<a href="https://colab.research.google.com/github/w86w86/econ8320_US-Census/blob/main/MAIN_0506_0440.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Call the API with my variables for my research

> List of some states "46 SD", "47 TN","10 DE", "2 AK","5 AR","24 MD","31 NE"

CBSA:

"46 SD",
* "43620 Sioux Falls, SD",

"47 TN"
* "16860 Chattanooga, TN-GA",
* "28940 Knoxville, TN",
* "34980": "Nashville-Davidson--Murfreesboro--Franklin, TN",
* "28700": "Kingsport-Bristol-Bristol, TN-VA",
* "32820": "Memphis, TN-MS-AR",
* "17300": "Clarksville, TN-KY",
* "27740": "Johnson City, TN",
* "17420": "Cleveland, TN",

"10 DE"
* "20100 Dover, DE",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "41540": "Salisbury, MD-DE",
*
* 0 city for "2 AK"

"5 AR",
* "22900 Fort Smith, AR-OK",
* "30780 Little Rock-North Little Rock-Conway, AR",
* "22220 Fayetteville-Springdale-Rogers, AR-MO",
* "38220 Pine Bluff, AR",
* "32820 Memphis, TN-MS-AR",
*
"24 MD",
* "25180": "Hagerstown-Martinsburg, MD-WV",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "12580": "Baltimore-Columbia-Towson, MD",
* "37980": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",
* "15680": "California-Lexington Park, MD",
* "41540": "Salisbury, MD-DE",

###CONSTANTES

In [ ]:
# (10) IMPORT LIBRARY
import requests
import pandas as pd
import regex as re
import time
from datetime import datetime as dt
import json
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# (20) CONSTANTES

#Dict of all states
#all_states = requests.get('https://api.census.gov/data/2024/cps/basic/jan/variables.json')
#states = json.loads(all_states.text)
#states_dict = states['variables']['STATE']['values']['item']
#states = [state for state in states_dict.keys()]

#All year of the study


###(10) CLASS API

In [ ]:
# (30) CLASS API and Decorator [log_activity2]

def log_activity2(func):
    def wrapper(*args, **kwargs):
        gdrivePath='/content/gdrive/MyDrive/project/'
        # Create a logger
        logger = logging.getLogger('activity_logger')
        logger.setLevel(logging.INFO)

        # Create a file handler and set the logging format
        file_handler = logging.FileHandler(gdrivePath + 'activity.log')
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        # Add the file handler to the logger
        logger.addHandler(file_handler)

        # Log the activity before calling the wrapped function
        logger.info(f'Activity: {func.__name__}')

        # Call the wrapped function
        return func(*args, **kwargs)

    return wrapper


class API:
  """This generic class API will handle at once all request to the CPS api
  How to?:
  > instanciate a variable at once, then use it to browse through all variables (not data) on the Census API json file
  > example api:    api=API()

  Properties:
  > allVars_dict (variable): a dict of all variables for each 'year' (key)
  > examples:
    - api.allVars_dict[2023]        * to se all the variables (dict) in 2023
    - 'GTCBSA' in api.allVars_dict[2024].keys() *                is <GTCBSA> in 2024?
    - api.allState_dict[2021]       * all stateID:description in year 2021"""

  def __init__(self) -> None:
    self.get()

  def get(self):
    #TEST
    #abc = requests.get('https://api.census.gov/data/2023/cps/basic/jan/variables.json')
    #print(abc)
    allVars_dict = {year_: year_vars for year_, year_vars in \
            zip(self.allYearList(), [json.loads(requests.get('https://api.census.gov/data/'+ str(year_) +'/cps/basic/jan/variables.json').text)['variables'] for year_ in self.allYearList()])}

  def store(self):
    pass

  @log_activity2
  def allStatesIdList(self, year:int=2023):
    return list(map(int, self.allVars_dict[year]['GESTFIPS']['values']['item'].keys())) #convert in int

  @log_activity2
  def allMonthList3TLA(self):
    month_dict = {1:"jan",2:"feb",3:"mar",4:"apr",5:"may",6:"jun",7:"jul",8:"aug",9: "sep",10: "oct",11:"nov",12:"dec"}
    #return next(key for key, value in month_dict.items() if value == m)
    return list(month_dict.values())

  def allYearList(self):
    return range(2010, 1+dt.now().year)
    #return range(2022, dt.now().year) # ... to 2023

  @log_activity2
  def getVarsNameFromUrl(self, url):
    return re.findall(r'([A-Z]+)', url)


api=API()

###(20) CLASS FileCompMix:

In [ ]:
class FileCompMix:
  """This class will track the which State Year and Month has already been processed
  so we are ending up re-download the same file, knowing that we have close to 10,000 calls
  to make with an average of 2.50 secs = ~ 7 hours in total.

  Hou to?:
  > track = FileCompMix(filename [optional])
        - if filename contains any char then the class will generate a new fileName_%m%d-%H%M%.
        - defaut is   ' MixCompStatus.csv '
  > two main methods:
        - track.record_state_of_state_year_month()   *   to store the info
        - track.is_record_state_of_state_year_month  *   to check if elt exist already
  """
  def __init__(self, fileName='', gdrivePath='/content/gdrive/MyDrive/project/') -> None:
    if not len(fileName) == 0:
      fileName  = dt.now().strftime("_%m%d-%H%M%S")
    self.gdrivePath = gdrivePath
    self.track_file = self.gdrivePath + 'MixCompStatus' + fileName + ".csv"
    self.set_df()
    self.eltInFileList = [list(i) for i in self.df.values]


  def set_df(self):
    """Read the NTFS file, and load it to a df so we can browse in in and download what has not been yet download
    """
    try:
      self.df = pd.read_csv(self.track_file, header=None)
    except Exception as e:
      self.df = pd.DataFrame([])
    self.hasCSVdata = self.df.shape[0]

  def numberOfelement(self):
    return self.df.shape[0]

  def record_state_of_state_year_month(self, element):
    self.df = pd.concat([self.df, pd.DataFrame([element])], ignore_index=True)
    #print (f'__record_state_of_state_year_month__ len: {self.df.shape[0]}')
    self.df.to_csv(self.track_file, mode='w', header=False, index=False) #not mode='a'
    self.hasCSVdata = True

  def is_record_state_of_state_year_month(self, element):
    if not self.hasCSVdata:
      #print ('--------------------------------#1F File emply, so will add')
      return False
    else:
      #print('______________________')
      #print(f'------- element: {element}')
      #print(f'------- self.eltInFileList: {self.eltInFileList}')
      #print('______________________')
      if (list(element) in self.eltInFileList):
        #print ('--------------------------------#2T elt already there, so not added, skipped')
        return True
      #print ('--------------------------------#3F, so will add')
      return False

###(40) CLASS MIXCOMP

In [ ]:
# (40) CLASS MIXCOMP
#Decorator to track API response time
def track_response_time(func):
    def wrapper(self, *args, **kwargs):
        start_time = time.time()
        result = func(self, *args, **kwargs)
        end_time = time.time()
        response_time = end_time - start_time
        self.response_time = response_time  # Store in the class
        print(f"Function: '{func.__name__}()'. Census API query took: {response_time} seconds")
        return result
    return wrapper


#Class for unit-request (All State, specific Year, specific Month)
class MixComp:
  """
This class will pull the Mix composition about the US population from one state.
Code	    Label
tabulate  Counts of instances
PEMNTVTY	Demographics-native country of mother
PEMARITL	Demographics-marital status
PESEX	    Demographics-sex
PRCITSHP	Demographics-United States citizenship group
GTCBSA	  Demographics-city level
STATE	    FIPS STATE Code
GTCBSA	  Demographics-city level

  """
  def __init__(self, year, month):
      #key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48' #w86
      key = '&key='+'a24a131b4da25b83199bdc7202fcab553b9e2983' #luos

      #data = pd.DataFrame(list(json.loads(requests.get("https://api.census.gov/data/2022/cps/basic/nov?get=PWSSWGT,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,PTIO1OCD,PTIO2OCD,PECYC,PEDIPGED,GTCBSA&for=state:*&key=a24a131b4da25b83199bdc7202fcab553b9e2983").text))[1:], \
      #                                                                columns = ['weight', 'nativity', 'marital', 'sex', 'citiz', 'occ1','occ2', 'collegcred','highsch', 'city', 'state'])

      self.url = 'https://api.census.gov/data/'+str(year)+'/cps/basic/'+month+'?get=PWSSWGT,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,PTIO1OCD,PTIO2OCD,PECYC,PEDIPGED,GTCBSA&for=state:*'+ key

      self.gdrivePath='/content/gdrive/MyDrive/project/'
      self.activityLog = self.gdrivePath + 'activity.log'
      self.year = year
      self.month = month
      self.response_time = 0 # how long the API took to run on Census CPS
      self.dataAPI = None
      self.dataText = None
      self.data4Df = []
      self.df = pd.DataFrame(None) #Will contain the final df(dataframe)
      self.initLogger()

  def initLogger(self):
      # Create a logger
      self.logger = logging.getLogger('activity_logger')
      self.logger.setLevel(logging.INFO)
      # Create a file handler and set the logging format
      file_handler = logging.FileHandler(self.activityLog)
      formatter = logging.Formatter('%(asctime)s - %(levelname)s - [w86] %(message)s')
      file_handler.setFormatter(formatter)
      # Add the file handler to the logger
      self.logger.addHandler(file_handler)

  def __str__(self):
      return f"""
      Mix Composition of the US population: month={self.month}, year={self.year}
      Dataframe head:
         {self.df.head(3)}"""

  def __repr__(self):
      return f"MixComp({self.year}, {self.month})"

  def __len__(self):
      return self.df.shape[0]

  def __add__(self, other):
      return pd.concat([self.df, other.df], axis=0)

  # to convert in class
  def checkVariable(self, year, url):
      """This function will check if a set of variables are declared on the Census API. Using the an url as input. Vars are in ALL_CAPS
      """
      api=API()
      api.get()
      matches = api.getVarsNameFromUrl(url)
      cps_vars = api.allVars_dict[year].keys()
      res = dict()
      for var in matches:
        res[var] = True if var in cps_vars else False
      res['status'] = all([status for status in res.values()])
      return res

  @track_response_time
  def get(self):
      #This section will validate all variables and replace the city's with the right one if year>=2024 (GTCBSA->CBSA)
      check = self.checkVariable(self.year, self.url)
      print (f"checkVariable() = {check}")
      if not check['status']:
        self.logger.info(f"> Class MixComp: / get() / BEFORE - Variables = {check}")
        if (self.year>=2024) and ('CBSA' not in check.keys()): #before 2024, City variable in 'GTCBSA', starting on 2024, City variable was 'CBSA'
          self.url = re.sub('GTCBSA', 'CBSA', self.url)
          self.logger.info(f"Class MixComp: / get() / > AFTER -  Variables = {self.checkVariable(self.year, self.url)}\nNew URL: {self.url}")
        else:
          self.logger.info(f"Class MixComp: / get() / > Please check all your Census variables. Unable to find some in the API call. Variables = {check}")

      try:
        # download the Demographics_Native_Country_Of_Mother for Mix-Compostion
        self.dataAPI = requests.get(self.url)
        res = dict({'response time in secs': round(self.response_time,2), 'status': True if 200==self.dataAPI.status_code else False, 'code':self.dataAPI.status_code, 'comment': 'url:' + self.url})
        self.logger.info(f'Class MixComp: / get() / > Return dict: [[[[ {res} ]]]]')
        return res

      except requests.exceptions.RequestException as e:
          # Handle connection errors or HTTP errors
          #raise("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")
          self.logger.info(f'Class MixComp: / get() / > STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[ {e} ]]]]')
          self.logger.info(f'Class MixComp: / get() / > Skipped !!')
      except AttributeError:
          self.logger.info(f'Class MixComp: / get() / > STOP ERROR: Please check your variable name!')

  def buildingDf(self):
      """
      ##Definition of variables:
      1. weight     represents the information related to population counts.
      2. nativity   column provides information about the state
      1. PEMNTVTY   Demographics-native country of mother
      3. marital    Demographics-marital status
      4. sex        Demographics-sex
      5. citiz      Demographics-United States citizenship group
      6. occ1       Primary Job
      7. occ2       Second Job
      8. collegcred college credit completed
      5. city       Demographics-city level
      We will understand the population composition by state about how mixed it is based on the native country of mothers,
      marital status distribution, gender demographics, citizenship status, at the city-level characteristics.
      This data can be used to uncover trends, patterns, and insights into the social and cultural fabric of the US population.
      """
      ##Build df and Renmane Df Columns for easy reading.
      self.df = pd.DataFrame(list(json.loads(self.dataAPI.text))[1:],columns = ['weight', 'nativity', 'marital', 'sex', 'citiz', 'occ1','occ2', 'collegcred','highsch', 'city', 'state'])
      ##Remove unknown City ID city==0
      self.df = self.df(self.df['city']!=0)
      self.logger.info(f'Class MixComp: / buildingDf > Remove unknown City ID, Build df and Renmane Df Columns for easy reading.')
      ##Building the YYYYMM column
      self.df['month'] = pd.Series([dt.strptime(str(self.month), "%b") for _ in range(self.df.shape[0])])
      self.df['year'] = pd.Series([dt.strptime(str(self.year), "%Y") for _ in range(self.df.shape[0])])
      self.logger.info(f'Class MixComp: / buildingDf > Built Dataframe for YYYYMM:{str(self.year)}-{str(self.month)}')
      #convert column in true type 'citiz', 'occ1','occ2', 'collegcred','highsch', 'city', 'state'])
      self.df['weight'] = self.df['weight'].astype(float)
      self.df['nativity'] = self.df['nativity'].astype(int)
      self.df['marital'] = self.df['marital'].astype(int)
      self.df['sex'] = self.df['sex'].astype(int)
      self.df['citiz'] = self.df['citiz'].astype(int)
      self.df['occ1'] = self.df['occ1'].astype(int)
      self.df['occ2'] = self.df['occ2'].astype(int)
      self.df['collegcred'] = self.df['collegcred'].astype(int)
      self.df['highsch'] = self.df['highsch'].astype(int)
      self.df['city'] = self.df['city'].astype(int)
      self.df['state'] = self.df['state'].astype(int)

  def save(self, file=''):
      try:
        if file == '':
          file = str(self.year)+'-'+ str(self.month)+'-MixComp.csv'
        r = self.df.to_csv( self.gdrivePath + file, index=False)
        self.logger.info(f'Class MixComp: / save > Created CSV file: {file}')
        return r
      except FileNotFoundError:
        raise (f'Check the path {file} of the file')
        self.logger.info(f'Class MixComp: / save > Check the path {file} of the file')
        self.logger.info(f'Class MixComp: / save > Skipped {file} creation')

  def main(self):
      self.logger.info(f"> {'v'*30} year:{self.year},month:{self.month} {'v'*30}")
      self.logger.info(f'Class MixComp: / main >  method - Entrance')
      self.logger.info(f'>Class MixComp: / main > get() method')

      url_call = self.get()
      self.logger.info(f'Class MixComp: / main - line: get() method - Result:{url_call}')

      try:
        if url_call['status']:
          self.buildingDf();  self.logger.info(f'Class MixComp: / main > URL found (200), now Creating the df')
          self.save();        self.logger.info(f'Class MixComp: / main > save() method')
        else:
          self.logger.info(f"Class MixComp: / main > Error during URL call, Code return: {url_call['code']}, Skipped !!")
      except Exception as e:
        self.logger.info(f"Class MixComp: / main > Error during URL call, Code return was undefinied, !! Please also check the API KEY!! Skipped !!")
      self.logger.info(f'Class MixComp: / main > Called main(self) method - End')
      self.logger.info(f"> {'^'*90}")
      return url_call['status']


###(60) CORE #1

In [ ]:
## (60 - 1) Going through All Months at once
track = FileCompMix()
count = {'initiallyInFile': track.numberOfelement(), 'processed': 0, 'skipped': 0}
print ('Begin>',count)

for year in api.allYearList():
  for month in api.allMonthList3TLA(): #['jan', 'feb']:
    elt = [year,month]
    if not track.is_record_state_of_state_year_month(elt):
      dataA = MixComp(*elt) #(year, month)

      result = dataA.main()

      if count['processed']%500==0: print ('+', end='')
      count['processed'] += 1
      if result: track.record_state_of_state_year_month(elt)
    else:
      count['skipped'] += 1
      if count['skipped']%1000==0: print ('.', end='')

print ('\nEnd  <',count)
print (f"count['processed']: {count['processed']},count['skipped']: {count['skipped']}")

###Merging to one

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import json
import re
import pandas  as pd

mergedJSON = {}
mergedJSON['merged'] = '/content/gdrive/MyDrive/mergedByYear/'
mergedJSON['unMergedPath'] = '/content/gdrive/MyDrive/project/'
mergedJSON['files'] ={}
mergedJSON['yearly'] = {}

def merge():
    """Will transfer all done file in a folder "mergedByYear"
    """
    filenameList = os.listdir(mergedJSON['unMergedPath'])
    filenameList = [file for file in filenameList if re.match(r'^\d{4}', file)]

    print (filenameList)
    "Example: 2010-apr-MixComp.csv"

    for file in filenameList:
      year = file.split('-')[0]
      if (int(year)>=2010) and (int(year)<=2024):
        if year not in mergedJSON['yearly']:
          mergedJSON['yearly'][year]=[]
        df = pd.read_csv(mergedJSON['unMergedPath'] + file)
        mergedJSON['yearly'][year].append(df)

    mergedJSON['fused'] = {}
    for year, dfs in mergedJSON['yearly'].items():
      mergedJSON['fused'][year] = pd.concat(dfs)

    for year, df in mergedJSON['fused'].items():
      path = mergedJSON['merged']
      df.to_csv(f'{path}{year}_merged.csv', index=False)

merge()

###Attempt to display graph

In [31]:
import os, regex as re, pandas as pd
mergedJSON = {}
mergedJSON['merged'] = '/content/gdrive/MyDrive/mergedByYear/'

filenameList = os.listdir(mergedJSON['merged'])
filenameList = [file for file in filenameList if re.match(r'^\d{4}', file)]
filenameList = iter(filenameList)
file = next(filenameList)
print (f'file:{file}')

df = pd.read_csv (mergedJSON['merged'] + file)
print (df.columns)
#cleaning / Should remove once taken care of them in the build4df method
df_Len1 = len(df) #initial length
df = df[df['city']!=0]
df_Len2 = len(df)
df = df[df['state']!='state']
df_Len3 = len(df)
print (f'Cleaning / df_Len1:{df_Len1}, df_Len2:{df_Len2}, df_Len3:{df_Len3}')
#Convert to write type / should be remove once I compile all
df['weight'] = df['weight'].astype(float)
df['nativity'] = df['nativity'].astype(int)
df['marital'] = df['marital'].astype(int)
df['sex'] = df['sex'].astype(int)
df['citiz'] = df['citiz'].astype(int)
df['occ1'] = df['occ1'].astype(int)
df['occ2'] = df['occ2'].astype(int)
df['collegcred'] = df['collegcred'].astype(int)
df['highsch'] = df['highsch'].astype(int)
df['city'] = df['city'].astype(int)
df['state'] = df['state'].astype(int)


file:2022_merged.csv


<ipython-input-31-2b7bc3424833>:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv (mergedJSON['merged'] + file)


Index(['weight', 'nativity', 'marital', 'sex', 'citiz', 'occ1', 'occ2',
       'collegcred', 'highsch', 'city', 'state', 'YYYYMM'],
      dtype='object')
Cleaning / df_Len1:1222915, df_Len2:1112858, df_Len3:1112846


In [34]:
grouped = df.groupby(['state', 'marital','YYYYMM'])['weight'].sum().reset_index()
grouped [grouped['state']==31]

#print (df)

,state,marital,YYYYMM,weight
2267,31,-1,2022-01-01,410439.5647
2268,31,-1,2022-02-01,269800.1775
2269,31,-1,2022-03-01,403846.6058
2270,31,-1,2022-04-01,258736.1278
2271,31,-1,2022-05-01,386664.7975
...,...,...,...,...
2346,31,6,2022-08-01,292228.1547
2347,31,6,2022-09-01,320483.4795
2348,31,6,2022-10-01,440080.1206
2349,31,6,2022-11-01,281399.4771
